<a href="https://colab.research.google.com/github/can-ishk/sokoban-ai-project/blob/main/sokoban.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction:
Sokoban Solver Implementation by our team.
References:

*   https://www.cs.huji.ac.il/w~ai/projects/2012/SokobanWP/files/report.pdf
*   https://arxiv.org/pdf/1807.00049.pdf
*   https://baldur.iti.kit.edu/theses/SokobanPortfolio.pdf
*   https://github.com/angusfung/sokoban-AI (for future)



In [ ]:
import os, sys
import copy
from collections import deque
import heapq
import math

# State Modelling:

In [ ]:
################################# Declaration: #################################
# Implementation inspired by https://arxiv.org/pdf/1807.00049.pdf
# Some parts are borrowed directly and then modified/upgraded as per requirements.
################################################################################
class Matrix(list):
    """
    Game state is represented using a matrix.
    It contains not only the grid information but also
    utility methods
    """
    size = None
    target_found = False
    _string = None
    moves = None
    actions = ""
    def getSize(self):
        """
        Gets the size of the matrix (The maximum width/height)
        """
        return self.size

    def getPlayerPosition(self):
        """
        Gets the position of the player in the current state
        """
        # Iterate all Rows
        for i in range(0, len(self)):
            # Iterate all columns
            for k in range(0, len(self[i]) - 1):
                if self[i][k] == "@":
                    return [k, i]

    def getBoxes(self):
        """
        Gets the position of all the boxes on screen
        """
        # Iterate all Rows
        boxes = []
        for i in range(0, len(self)):
            # Iterate all columns
            for k in range(0, len(self[i]) - 1):
                if self[i][k] == "$":
                    boxes.append([k, i])
        return boxes

    def getTargets(self):
        """
        Gets the position of all the targets on screen
        """
        # Iterate all Rows
        boxes = []
        for i in range(0, len(self)):
            # Iterate all columns
            for k in range(0, len(self[i]) - 1):
                if self[i][k] == ".":
                    boxes.append([k, i])
        return boxes

    def isSuccess(self):
        """
        Checks if the current state is the end state of the game.
        """
        return len(self.getBoxes()) == 0
        
    def getPossibleActions(self):
        x = self.getPlayerPosition()[0]
        y = self.getPlayerPosition()[1]
        def update_valid(item, move, get_two_step):
            if item not in "*#$":
                return (move.lower(), 'Move')
            if item in "$*" and get_two_step() not in "*#$":
                # We really prefer pushing the blocks over just roaming around
                # We do not like moving blocks out of their respective targets
                return (move, "Push") if item == '$' else (move, "PushOut")
            return None
        moves = []
        action_cost = update_valid(self[y][x - 1], 'L', lambda: self[y][x - 2])
        if action_cost is not None:
            moves.append(action_cost)
        action_cost = update_valid(self[y][x + 1], 'R', lambda: self[y][x + 2])
        if action_cost is not None:
            moves.append(action_cost)
        action_cost = update_valid(self[y - 1][x], 'U', lambda: self[y - 2][x])
        if action_cost is not None:
            moves.append(action_cost)
        action_cost = update_valid(self[y + 1][x], 'D', lambda: self[y + 2][x])
        if action_cost is not None:
            moves.append(action_cost)
        return moves

    def successor(self, direction, performOnSelf=False):
        if performOnSelf:
            return self.successorInternal(self, direction)
        matrix = copy.deepcopy(self)
        self.successorInternal(matrix, direction)
        matrix._string = None
        return matrix

    def toString(self, clear=False):
        """
        Gives a string version of self that is cached with the assumption that
        the object has not been mutated. The value can be stale if clear is not true
        """
        if self._string is not None and clear is False:
            return self._string
        self._string = "\n".join(["".join(x) for x in self])
        return self._string

    def __hash__(self):
        return hash(self.toString())

    def __str__(self):
        return self.toString()

    def successorInternal(self, matrix, direction):
        x = matrix.getPlayerPosition()[0]
        y = matrix.getPlayerPosition()[1]

        # print boxes
        # print matrix.getBoxes()

        if direction.upper() == "L":
            # print "######### Moving Left #########"

            # if is_space
            if matrix[y][x - 1] == " ":
                # print "OK Space Found"
                matrix[y][x - 1] = "@"
                if matrix.target_found == True:
                    matrix[y][x] = "."
                    matrix.target_found = False
                else:
                    matrix[y][x] = " "

            # if is_box
            elif matrix[y][x - 1] == "$":
                # print "Box Found"
                if matrix[y][x - 2] == " ":
                    matrix[y][x - 2] = "$"
                    matrix[y][x - 1] = "@"
                    if matrix.target_found == True:
                        matrix[y][x] = "."
                        matrix.target_found = False
                    else:
                        matrix[y][x] = " "
                elif matrix[y][x - 2] == ".":
                    matrix[y][x - 2] = "*"
                    matrix[y][x - 1] = "@"
                    if matrix.target_found == True:
                        matrix[y][x] = "."
                        matrix.target_found = False
                    else:
                        matrix[y][x] = " "

            # if is_box_on_target
            elif matrix[y][x - 1] == "*":
                # print "Box on target Found"
                if matrix[y][x - 2] == " ":
                    matrix[y][x - 2] = "$"
                    matrix[y][x - 1] = "@"
                    if matrix.target_found == True:
                        matrix[y][x] = "."
                    else:
                        matrix[y][x] = " "
                    matrix.target_found = True

                elif matrix[y][x - 2] == ".":
                    matrix[y][x - 2] = "*"
                    matrix[y][x - 1] = "@"
                    if matrix.target_found == True:
                        matrix[y][x] = "."
                    else:
                        matrix[y][x] = " "
                    matrix.target_found = True

            # if is_target
            elif matrix[y][x - 1] == ".":
                # print "Target Found"
                matrix[y][x - 1] = "@"
                if matrix.target_found == True:
                    matrix[y][x] = "."
                else:
                    matrix[y][x] = " "
                matrix.target_found = True

            # else
            else:
                pass
                #print "There is a wall here"

        elif direction.upper() == "R":
            # print "######### Moving Right #########"

            # if is_space
            if matrix[y][x + 1] == " ":
                # print "OK Space Found"
                matrix[y][x + 1] = "@"
                if matrix.target_found == True:
                    matrix[y][x] = "."
                    matrix.target_found = False
                else:
                    matrix[y][x] = " "

            # if is_box
            elif matrix[y][x + 1] == "$":
                # print "Box Found"
                if matrix[y][x + 2] == " ":
                    matrix[y][x + 2] = "$"
                    matrix[y][x + 1] = "@"
                    if matrix.target_found == True:
                        matrix[y][x] = "."
                        matrix.target_found = False
                    else:
                        matrix[y][x] = " "

                elif matrix[y][x + 2] == ".":
                    matrix[y][x + 2] = "*"
                    matrix[y][x + 1] = "@"
                    if matrix.target_found == True:
                        matrix[y][x] = "."
                        matrix.target_found = False
                    else:
                        matrix[y][x] = " "

            # if is_box_on_target
            elif matrix[y][x + 1] == "*":
                # print "Box on target Found"
                if matrix[y][x + 2] == " ":
                    matrix[y][x + 2] = "$"
                    matrix[y][x + 1] = "@"
                    if matrix.target_found == True:
                        matrix[y][x] = "."
                    else:
                        matrix[y][x] = " "
                    matrix.target_found = True

                elif matrix[y][x + 2] == ".":
                    matrix[y][x + 2] = "*"
                    matrix[y][x + 1] = "@"
                    if matrix.target_found == True:
                        matrix[y][x] = "."
                    else:
                        matrix[y][x] = " "
                    matrix.target_found = True

            # if is_target
            elif matrix[y][x + 1] == ".":
                # print "Target Found"
                matrix[y][x + 1] = "@"
                if matrix.target_found == True:
                    matrix[y][x] = "."
                else:
                    matrix[y][x] = " "
                matrix.target_found = True

            # else
            else:
                pass
                # print "There is a wall here"

        elif direction.upper() == "D":
            # print "######### Moving Down #########"

            # if is_space
            if matrix[y + 1][x] == " ":
                # print "OK Space Found"
                matrix[y + 1][x] = "@"
                if matrix.target_found == True:
                    matrix[y][x] = "."
                    matrix.target_found = False
                else:
                    matrix[y][x] = " "

            # if is_box
            elif matrix[y + 1][x] == "$":
                # print "Box Found"
                if matrix[y + 2][x] == " ":
                    matrix[y + 2][x] = "$"
                    matrix[y + 1][x] = "@"
                    if matrix.target_found == True:
                        matrix[y][x] = "."
                        matrix.target_found = False
                    else:
                        matrix[y][x] = " "

                elif matrix[y + 2][x] == ".":
                    matrix[y + 2][x] = "*"
                    matrix[y + 1][x] = "@"
                    if matrix.target_found == True:
                        matrix[y][x] = "."
                        matrix.target_found = False
                    else:
                        matrix[y][x] = " "

            # if is_box_on_target
            elif matrix[y + 1][x] == "*":
                # print "Box on target Found"
                if matrix[y + 2][x] == " ":
                    matrix[y + 2][x] = "$"
                    matrix[y + 1][x] = "@"
                    if matrix.target_found == True:
                        matrix[y][x] = "."
                    else:
                        matrix[y][x] = " "
                    matrix.target_found = True

                elif matrix[y + 2][x] == ".":
                    matrix[y + 2][x] = "*"
                    matrix[y + 1][x] = "@"
                    if matrix.target_found == True:
                        matrix[y][x] = "."
                    else:
                        matrix[y][x] = " "
                    matrix.target_found = True

            # if is_target
            elif matrix[y + 1][x] == ".":
                # print "Target Found"
                matrix[y + 1][x] = "@"
                if matrix.target_found == True:
                    matrix[y][x] = "."
                else:
                    matrix[y][x] = " "
                matrix.target_found = True

            # else
            else:
                pass
                # print "There is a wall here"

        elif direction.upper() == "U":
            # print "######### Moving Up #########"

            # if is_space
            if matrix[y - 1][x] == " ":
                # print "OK Space Found"
                matrix[y - 1][x] = "@"
                if matrix.target_found == True:
                    matrix[y][x] = "."
                    matrix.target_found = False
                else:
                    matrix[y][x] = " "

            # if is_box
            elif matrix[y - 1][x] == "$":
                # print "Box Found"
                if matrix[y - 2][x] == " ":
                    matrix[y - 2][x] = "$"
                    matrix[y - 1][x] = "@"
                    if matrix.target_found == True:
                        matrix[y][x] = "."
                        matrix.target_found = False
                    else:
                        matrix[y][x] = " "

                elif matrix[y - 2][x] == ".":
                    matrix[y - 2][x] = "*"
                    matrix[y - 1][x] = "@"
                    if matrix.target_found == True:
                        matrix[y][x] = "."
                        matrix.target_found = False
                    else:
                        matrix[y][x] = " "

            # if is_box_on_target
            elif matrix[y - 1][x] == "*":
                # print "Box on target Found"
                if matrix[y - 2][x] == " ":
                    matrix[y - 2][x] = "$"
                    matrix[y - 1][x] = "@"
                    if matrix.target_found == True:
                        matrix[y][x] = "."
                    else:
                        matrix[y][x] = " "
                    matrix.target_found = True

                elif matrix[y - 2][x] == ".":
                    matrix[y - 2][x] = "*"
                    matrix[y - 1][x] = "@"
                    if matrix.target_found == True:
                        matrix[y][x] = "."
                    else:
                        matrix[y][x] = " "
                    matrix.target_found = True

            # if is_target
            elif matrix[y - 1][x] == ".":
                # print "Target Found"
                matrix[y - 1][x] = "@"
                if matrix.target_found == True:
                    matrix[y][x] = "."
                else:
                    matrix[y][x] = " "
                matrix.target_found = True

            # else
            else:
                pass
                # print "There is a wall here"
        

# Level Modelling:

In [ ]:

class Level:
    matrix = Matrix()
    matrix_history = []

    def __init__(self,set,level_num):

        del self.matrix[:]
        del self.matrix_history[:]
        # Create level
        with open(set+'/level' + str(level_num)+'.xsb', 'r') as f:
                for row in f.read().splitlines():
                    self.matrix.append(list(row))

        max_row_length = 0
        # Iterate all Rows
        for i in range(0, len(self.matrix)):
            # Iterate all columns
            row_length = len(self.matrix[i])
            if row_length > max_row_length:
                max_row_length = row_length
        self.matrix.size = [max_row_length, len(self.matrix)]
        self.matrix.width = max_row_length
        self.matrix.height = len(self.matrix)

    def __del__(self):
        "Destructor to make sure object shuts down, etc."

    def getMatrix(self):
        return self.matrix

    def addToHistory(self,matrix):
        self.matrix_history.append(copy.deepcopy(matrix))

    def undo(self):
        if len(self.matrix_history) > 0:
            lastMatrix = self.matrix_history.pop()
            self.matrix = lastMatrix
            return lastMatrix
        else:
            return self.matrix




# Game Solver Engine:

> Methods available: DFS, BFS, UCS (with two costs), A* (with two heuristics).


> Implemention: A* implemented. UCS is implemented as A* with no heuristic. BFS is implemented as UCS with no cost. DFS is implemented seperately. 





In [ ]:
def distance(method):
    def calc(state, cache):
        if 'min_distance' not in cache:
            cache['min_distance'] = {}
        player = state.getPlayerPosition()
        boxes = state.getBoxes()
        targets = state.getTargets()
        total = 0
        key = (",".join([str(x[0]) + "-" + str(x[1]) for x in boxes]),
               ",".join([str(x[0]) + "-" + str(x[1]) for x in targets]))
        if key in cache['min_distance']:
            total = cache['min_distance'][key]
        else:
            for b in boxes:
                total += min([method(b, t) for t in targets] or [0])
            cache['min_distance'][key] = total
        total += sum([method(player, b) for b in boxes] or [0])
        return total

    return calc

def default(key, cache):
    if key == 'Move':
        return 1
    elif key == 'Push':
        return 2
    elif key == 'PushOut':
        return 10


def cost2(key, cache):
    if key == 'Move':
        return 2
    elif key == 'Push':
        return 1
    elif key == 'PushOut':
        return 2


class solver():
    cache = {}
    costs = {
        "none": lambda key, cache: 1,
        "default": default,
        "cost2": cost2
    }
    global distance
    heuristic = {
        "manhatten": distance(lambda a, b: abs(a[0] - b[0]) + abs(a[1] - b[1])),
        "none": lambda x, y: 0,
        "euclidean": distance(lambda x, y: math.sqrt((x[0]-y[0])**2 + (x[1]-y[1])**2)),
    }
    def refresh(self):
        self.cache = {}

    def dfs(self, startState, maxDepth=150, cache={}):
        stack = deque([(startState, "")])
        while len(stack) > 0:
            state, actions = stack.pop()
            cache[state.toString()] = len(actions)
            # print(actions)
            if state.isSuccess():
                return (actions,len(cache))
            if len(actions) == maxDepth:
                continue
            for (action, _) in state.getPossibleActions():
                successor = state.successor(action)
                # Don't go to an explored state
                if successor.toString() in cache and cache[successor.toString()] <= len(actions) + 1:
                    continue
                # Don't go to a state already marked for visiting
                stack.append((successor, actions + action))
        return ("",0)

    def bfs(self, startState, maxDepth=float('inf'), cache={}):
        return self.ucs(startState, cache=cache, cost="none")

    def ucs(self, startState, cost="default", maxCost=500, cache={}):
        return self.astar(startState, cost=cost, maxCost=maxCost, cache=cache, heuristic="none")

    def astar(self, startState, maxCost=1000, cost="default", heuristic="manhatten", cache={}):
        h = self.heuristic[heuristic]
        costCalc = self.costs[cost]
        queue = PriorityQueue()
        action_map = {}
        startState.h = h(startState, self.cache)
        queue.update(startState, startState.h)
        action_map[startState.toString()] = ""
        while not queue.empty():
            state, cost = queue.removeMin()
            actions = action_map[state.toString()]
            cache[state.toString()] = len(actions)
            # print(actions)
            if state.isSuccess():
                return (actions,len(cache))
            else
            if cost >= maxCost:
                continue
            for (action, cost_delta) in state.getPossibleActions():
                successor = state.successor(action)
                if successor.toString() in cache:
                    continue # = Don't go to an explored state again
                old = action_map[successor.toString()] if successor.toString(
                ) in action_map else None
                if not old or len(old) > len(actions) + 1:
                    action_map[successor.toString()] = actions + action
                successor.h = h(successor, self.cache)
                queue.update(successor, cost + costCalc(cost_delta, self.cache) + successor.h - state.h)
        return ("",0)

def manhattenDistance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


# Data structure for supporting uniform cost search.
class PriorityQueue:
    def __init__(self):
        self.DONE = -100000
        self.heap = []
        self.priorities = {}  # Map from state to priority

    # Insert |state| into the heap with priority |newPriority| if
    # |state| isn't in the heap or |newPriority| is smaller than the existing
    # priority.
    # Return whether the priority queue was updated.
    def update(self, state, newPriority):
        oldPriority = self.priorities.get(state)
        if oldPriority == None or newPriority < oldPriority:
            self.priorities[state] = newPriority
            heapq.heappush(self.heap, (newPriority, state))
            return True
        return False

    # Returns (state with minimum priority, priority)
    # or (None, None) if the priority queue is empty.
    def removeMin(self):
        while len(self.heap) > 0:
            priority, state = heapq.heappop(self.heap)
            if self.priorities[state] == self.DONE:
                continue  # Outdated priority, skip
            self.priorities[state] = self.DONE
            return (state, priority)
        return (None, None)  # Nothing left...

    def empty(self):
        return len(self.heap) == 0


# Level Loader/Game Runner:

In [ ]:
def movePlayer(direction,myLevel):

    matrix = myLevel.getMatrix()

    myLevel.addToHistory(matrix)

    matrix.successor(direction, True)

    if matrix.isSuccess():
        global current_level
        current_level += 1
        initLevel(level_set,current_level)

def initLevel(level_set,level):
    # Create an instance of this Level
    global myLevel
    myLevel = Level(level_set,level)


def runGame(args):

    global current_level
    current_level = args['level']
    global level_set
    level_set = args['level_set']


    # Level Initialisation
    if current_level==args['last_level']+1:
        sys.exit()
    initLevel(level_set,current_level)
    count=0
    old_level = current_level - 1
    while old_level == current_level - 1:
        if current_level==args['last_level']+1:
            sys.exit()
        old_level = current_level
        moves = solve(args, myLevel)
        if moves != "":
            for move in moves:
                movePlayer(move, myLevel)
        else:
            print ("Failed for level %d"%(current_level))

            current_level = current_level + 1
            if current_level==args['last_level']+1:
                sys.exit()
            else:
                initLevel(level_set,current_level)

def solveInternal(cache, method, cost, heuristic):
    solution = solver()
    solution.refresh()
    moves = []
    moves_cache=[]
    if method == "dfs":
        moves_cache = solution.dfs(myLevel.getMatrix(), cache=cache)
    elif method == "bfs":
        moves_cache = solution.bfs(myLevel.getMatrix(), cache=cache)
    elif method == "ucs":
        moves_cache = solution.ucs(myLevel.getMatrix(), cache=cache)
    elif method == "astar":
        moves_cache = solution.astar(myLevel.getMatrix(), cache=cache, cost=cost, heuristic=heuristic)
    return moves_cache

def solve(args, myLevel):
    moves_cache = solveInternal(method=args['method'], cache={}, cost=args['cost'], heuristic=args['heuristic'])
    print ("Level: %d, Moves: %s Length: %d States Explored: %d" % (current_level, moves_cache[0], len(moves_cache[0]), moves_cache[1]))
    return moves_cache[0]

# def default(str):
#   return str + ' [Default: %default]'


# **Test Examples Ahead: (Easier Test Cases)**

In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 1,
    'last_level': 1,
    'level_set':'test_examples',
    # 'level_set':'project_levels'
}
runGame(args)

KeyboardInterrupt: ignored

In [ ]:
  
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 2,
    'last_level': 2,
    'level_set':'test_examples',
    # 'level_set':'project_levels'
}
runGame(args)

Level: 2, Moves: drrrrdrrddllUUruLLLulldRddrRlluuRRRdrrddlldlUrrruulluurDldDLddrUUUddrruuLuLDDuuLLulldRddRRdRUUruulDlLulDrrrDDlddrUUUrrddLruuluLLLrrddlddrUUUruLL Length: 144 States Explored: 43868


SystemExit: ignored

In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 3,
    'last_level': 3,
    'level_set':'test_examples',
    # 'level_set':'project_levels'
}
runGame(args)

Level: 3, Moves: lluRRRRRllldlddddrrruurruuruulDlldLLulDDDDurUUrrurrDrddlllddlldlluRuuuuuRRRRldllddldddrruruurrruulDrdLLruuruulDDDrdLuuulldllddlddRRdrU Length: 134 States Explored: 582527


SystemExit: ignored

In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 4,
    'last_level': 4,
    'level_set':'test_examples',
    # 'level_set':'project_levels'
}
runGame(args)

Level: 4, Moves: dlldddllluurRllddrrrurrrruuuuuulllllldddRDrrruullDldRluluuurrrrrrddddddllllULrddlUUrdrrrruuuuuulllllldddrrDDldRuuuurrddLruulldlDDrUdddllluuRRllddrrUrUdldlluurRuuluuurrrrrrddddddllL Length: 180 States Explored: 201583


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 5,
    'last_level': 5,
    'level_set':'test_examples',
    # 'level_set':'project_levels'
}
runGame(args)

Level: 5, Moves: rurrrdrruuuLulDDDllldlluRRRRdRRdrUllUUUruulDllLdlDururrrDDDllldlluRRRRdRUUUddlllluuruulDDDururrRdddllldlluRRRRdrRdrruLLLUlllluururrrdrrdDuulluurDlllldlddrrrrUURuLLLrrdddlllluuuurDrrrddddrdrruLuuruLLullllldddrrrruUruLLLLulDDDuurrrrddddRdrUUUruLLrdddlluuUddllldlluRuuurrrRdddllLdlUUU Length: 281 States Explored: 1247051


SystemExit: ignored

In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 6,
    'last_level': 6,
    'level_set':'test_examples',
    # 'level_set':'project_levels'
}
runGame(args)

# **Original Test Cases Ahead:**

In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 1,
    'last_level': 1,
    # 'level_set':'test_examples',
    'level_set':'test_cases'
}
runGame(args)

In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 2,
    'last_level': 2,
    # 'level_set':'test_examples',
    'level_set':'test_cases'
}
runGame(args)

In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 3,
    'last_level': 3,
    # 'level_set':'test_examples',
    'level_set':'test_cases'
}
runGame(args)

In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 4,
    'last_level': 4,
    # 'level_set':'test_examples',
    'level_set':'test_cases'
}
runGame(args)

In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 5,
    'last_level': 5,
    # 'level_set':'test_examples',
    'level_set':'test_cases'
}
runGame(args)

In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 6,
    'last_level': 6,
    # 'level_set':'test_examples',
    'level_set':'test_cases'
}
runGame(args)

In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 7,
    'last_level': 7,
    # 'level_set':'test_examples',
    'level_set':'test_cases'
}
runGame(args)

In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 8,
    'last_level': 8,
    # 'level_set':'test_examples',
    'level_set':'test_cases'
}
runGame(args)

In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 9,
    'last_level': 9,
    # 'level_set':'test_examples',
    'level_set':'test_cases'
}
runGame(args)

In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 10,
    'last_level': 10,
    # 'level_set':'test_examples',
    'level_set':'test_cases'
}
runGame(args)

# **Our Levels:**

In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 1,
    'last_level': 21,
    # 'level_set':'test_examples',
    'level_set':'project_levels'
}
runGame(args)

Level: 1, Moves: ulluururrdLDuulldddrdrruLLdlUrruuL Length: 34 States Explored: 583
Level: 2, Moves: rruullluuRlddrrrddllUdrruuuruullDllddRRlddrruUllluurrurrddrddLruuluulldDRdL Length: 75 States Explored: 4500
Level: 3, Moves: rDuurDDlLulDuulDD Length: 17 States Explored: 295
Level: 4, Moves: lllluUUUUdddlldlUUUUUrrRRRRurDD Length: 31 States Explored: 9801
Level: 5, Moves: DldRRdrUlluuRuulDDDldRR Length: 23 States Explored: 3702
Level: 6, Moves: DDDurrdLLuuurDlddrruLdlUlldRRuruulDrdLdlddrUUUrrdLulDlddrUluluRRRdLulDDruuruulDDrdLulD Length: 86 States Explored: 9108
Level: 7, Moves: RuulDDrddlluRRdDldR Length: 19 States Explored: 651
Level: 8, Moves: RuulDDrddlluRRdDldR Length: 19 States Explored: 651
Level: 9, Moves: DurDlDDuRDDLrrdLL Length: 17 States Explored: 582
Level: 10, Moves: rdrRRlluurDlddRRuLdlUrrRRRRlllDDldRRRRdrU Length: 41 States Explored: 78999
Level: 11, Moves: ulluururrdLDuulldddrdrruLLdlUrruuL Length: 34 States Explored: 583
Level: 12, Moves: rrurRdrURurrdLLLLLLdlluRRRRu

FileNotFoundError: ignored

In [ ]:
args = {
    'method':'astar',
    'cost': 'default',
    'heuristic': 'manhatten',
    'level': 1,
    'last_level': 3,
    # 'level_set':'test_examples',
    'level_set':'project_levels'
}
runGame(args)

Level: 1, Moves: ulluururrdLDuulldddrdrruLLdlUrruuL Length: 34 States Explored: 583
Level: 2, Moves: rruullluuRlddrrrddllUdrruuuruullDllddRRlddrruUllluurrurrddrddLruuluulldDRdL Length: 75 States Explored: 4500
Level: 3, Moves: rDuurDDlLulDuulDD Length: 17 States Explored: 295


SystemExit: ignored